In [1]:
import numpy as np
import pandas as pd

from molloader import *
from ML_utils import *

import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split, KFold, ParameterGrid

In [2]:
def drop_duplicates(df):
    df = df.drop_duplicates(subset="compound", keep="first")
    print("Number of molecules", df.shape[0])
    return df

In [3]:
def get_all_elements(df):
      return np.unique(np.hstack(np.array(df.species)))

In [4]:
class VoxelNet(nn.Module):
    def __init__(self, in_channels=1):
        super().__init__()
        # (N, C, D, H, W)
        self.conv1 = nn.Sequential(
            nn.Conv3d(in_channels, 16, 3, padding=0, stride=1),
            nn.SELU(0.3),
            nn.MaxPool3d(2))
        
        self.conv2 = nn.Sequential(
            nn.Conv3d(16, 16, 3, padding=0, stride=1),
            nn.SELU(),
            nn.MaxPool3d(2))
        
        self.conv3 = nn.Sequential(
            nn.Conv3d(16, 16, 3, padding=0, stride=1),
            nn.SELU())
        
        self.regressor = nn.Sequential(nn.Flatten(),
                                        nn.Linear(1024, 32),
                                        nn.SELU(),
                                        nn.Linear(32, 8),
                                        nn.SELU(),
                                        nn.Linear(8, 1))
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.regressor(x)
        return x

In [5]:
data_name = "all_221_cp5_3_species_oxides"
fig_folder = "plots_SELU/"

In [6]:
df = pd.read_pickle(f"data/{data_name}.pkl")
df = drop_duplicates(df)
df

Number of molecules 1983


auid  \
0     aflow:b778fabdb1b01db1   
2     aflow:272cd0e74db84a51   
4     aflow:7e6bdbc495310b44   
6     aflow:c0e1c25976bb4c75   
8     aflow:3b79b8b28a5d5cc1   
...                      ...   
3947  aflow:e243d024b0749ac0   
3949  aflow:ae4a07b2d68c2933   
3951  aflow:0d9d3ce033a32aae   
3953  aflow:627232532ed2fb40   
3955  aflow:2b3ea96a80ac45cd   

                                                   aurl  compound composition  \
0     aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgAlO/T00...  Ag1Al1O3   [1, 1, 3]   
2     aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgAsO/T00...  Ag1As1O3   [1, 1, 3]   
4     aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgAuO/T00...  Ag1Au1O3   [1, 1, 3]   
6     aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgB_hO/T0...   Ag1B1O3   [1, 1, 3]   
8     aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgBa_svO/...  Ag1Ba1O3   [1, 1, 3]   
...                                                 ...       ...         ...   
3947  aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/OXeZn/T00...  O3Xe1Zn1   [3, 1, 1]   
3949  aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/OXeZr_sv/...  O3Xe1Zr1   [3, 1, 1]   
3951  aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/OY_svZn/T...   O3Y1Zn1   [3, 1, 1]   
3953  aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/OY_svZr_s...   O3Y1Zr1   [3, 1, 1]   
3955  aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/OZnZr_sv/...  O3Zn1Zr1   [3, 1, 1]   

          species natoms spacegroup pearson_symbol  \
0     [Ag, Al, O]      5        221            cP5   
2     [Ag, As, O]      5        221            cP5   
4     [Ag, Au, O]      5        221            cP5   
6      [Ag, B, O]      5        221            cP5   
8     [Ag, Ba, O]      5        221            cP5   
...           ...    ...        ...            ...   
3947  [O, Xe, Zn]      5        221            cP5   
3949  [O, Xe, Zr]      5        221            cP5   
3951   [O, Y, Zn]      5        221            cP5   
3953   [O, Y, Zr]      5        221            cP5   
3955  [O, Zn, Zr]      5        221            cP5   

                                               geometry  \
0     [3.7688035, 3.7688035, 3.7688035, 90.0, 90.0, ...   
2     [3.7951634, 3.7951634, 3.7951634, 90.0, 90.0, ...   
4     [4.1003175, 4.1003175, 4.1003175, 90.0, 90.0, ...   
6      [3.484261, 3.484261, 3.484261, 90.0, 90.0, 90.0]   
8     [4.7271734, 4.7271734, 4.7271734, 90.0, 90.0, ...   
...                                                 ...   
3947   [4.370543, 4.370543, 4.370543, 90.0, 90.0, 90.0]   
3949   [4.322477, 4.322477, 4.322477, 90.0, 90.0, 90.0]   
3951  [4.3517566, 4.3517566, 4.3517566, 90.0, 90.0, ...   
3953   [4.155165, 4.155165, 4.155165, 90.0, 90.0, 90.0]   
3955   [3.723766, 3.723766, 3.723766, 90.0, 90.0, 90.0]   

                                   positions_fractional  \
0     [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5], [0.0, 0.5, ...   
2     [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5], [0.0, 0.5, ...   
4     [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5], [0.0, 0.5, ...   
6     [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5], [0.0, 0.5, ...   
8     [[-0.0, 0.0, -0.0], [0.5, 0.5, 0.5], [-0.0, 0....   
...                                                 ...   
3947  [[0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, ...   
3949  [[0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, ...   
3951  [[0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, ...   
3953  [[0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, ...   
3955  [[0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, ...   

                                    positions_cartesian  enthalpy_atom  \
0     [[0.0, 0.0, 0.0], [1.8844, 1.8844, 1.8844], [0...       -5.04863   
2     [[0.0, 0.0, 0.0], [1.89758, 1.89758, 1.89758],...       -4.55519   
4     [[0.0, 0.0, 0.0], [2.05016, 2.05016, 2.05016],...       -2.80408   
6     [[0.0, 0.0, 0.0], [1.74213, 1.74213, 1.74213],...       -4.55549   
8     [[-0.0, 0.0, -0.0], [2.36359, 2.36359, 2.36359...       -3.52096   
...                                                 ...            ...   
3947  [[0.0, 2.18527, 2.18527], [2.18527, 0.0, 2.185...       -1.89960   
3949  [

In [7]:
elements = get_all_elements(df)
nchannel = len(elements)
nchannel, elements

(79,
 array(['Ag', 'Al', 'As', 'Au', 'B', 'Ba', 'Be', 'Bi', 'Br', 'C', 'Ca',
        'Cd', 'Ce', 'Cl', 'Co', 'Cr', 'Cs', 'Cu', 'Dy', 'Er', 'Eu', 'F',
        'Fe', 'Ga', 'Gd', 'Ge', 'Hf', 'Hg', 'Ho', 'I', 'In', 'Ir', 'K',
        'La', 'Li', 'Lu', 'Mg', 'Mn', 'Mo', 'N', 'Na', 'Nb', 'Nd', 'Ni',
        'O', 'Os', 'P', 'Pa', 'Pb', 'Pd', 'Pr', 'Pt', 'Pu', 'Rb', 'Re',
        'Rh', 'Ru', 'S', 'Sb', 'Sc', 'Se', 'Si', 'Sn', 'Sr', 'Ta', 'Tb',
        'Tc', 'Te', 'Th', 'Ti', 'Tl', 'U', 'V', 'W', 'Xe', 'Y', 'Yb', 'Zn',
        'Zr'], dtype='<U2'))

In [8]:
net = VoxelNet(nchannel)
count_parameters(net)

81073

##  Cartesian

In [9]:
sigma = 0.1
L = 12.8
N = 32
epochs = 50
device = get_default_device()


seed_everything()
train_df, test_df = train_test_split(df, test_size=0.1)
print(train_df.shape, test_df.shape)

seed_everything()

net = VoxelNet(nchannel)
net = net.to(device)

ml_train = MolLoader(train_df, sigma=sigma, elements=elements,
               L=L, N=N, batch_size=64, nchannel=nchannel,
               shuffle=True, rotate_randomly=True, device=device, reduce_data=True, mode='cartesian')

ml_test = MolLoader(test_df, sigma=sigma, elements=elements,
               L=L, N=N, batch_size=128, nchannel=nchannel,
               shuffle=False, rotate_randomly=False, device=device, reduce_data=True, mode='cartesian')

fit(epochs, net, ml_train, ml_test, torch.optim.Adam, lr=0.001, weight_decay=0.00)

(1784, 13) (199, 13)
Initialised MolLoader with 1784 compounds.
    sigma=0.1, L=12.8, N=32, nchannel=79, mode=cartesian, shuffle=True, rotate=True, device=cuda
Initialised MolLoader with 199 compounds.
    sigma=0.1, L=12.8, N=32, nchannel=79, mode=cartesian, shuffle=False, rotate=False, device=cuda
Epoch 0: train loss 9.042786598205566 val loss 3.4420368671417236
Epoch 1: train loss 2.2276499271392822 val loss 2.301090717315674
Epoch 2: train loss 2.0687930583953857 val loss 2.1955337524414062
Epoch 3: train loss 1.8766776323318481 val loss 1.9986697435379028
Epoch 4: train loss 1.3838999271392822 val loss 1.1128392219543457
Epoch 5: train loss 0.6126717925071716 val loss 0.6515576243400574
Epoch 6: train loss 0.3997688591480255 val loss 0.6038097739219666
Epoch 7: train loss 0.42647045850753784 val loss 0.5344315767288208
Epoch 8: train loss 0.39392220973968506 val loss 0.4994639456272125
Epoch 9: train loss 0.33262431621551514 val loss 0.48317447304725647
Epoch 10: train loss 0.334

KeyboardInterrupt: 

### Train Plots

In [ ]:
# 0.1146
ml_pred = ml_train
ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=False)
ys, y_hats = predict(net, ml_pred)
plot_predictions(ys, y_hats)
plt.show()

In [ ]:
# 0.0727
# rotations
seed_everything()
ml_pred = ml_train
ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=True)
ys, y_hats = predict_epochs(net, ml_pred, epochs=5)
plot_predictions(ys, y_hats, alpha=0.01)
plt.show()

### Test Plots

In [ ]:
# 0.3196
ml_pred = ml_test
ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=False)
ys, y_hats = predict(net, ml_pred)
plot_predictions(ys, y_hats)
plt.show()

In [ ]:
# 0.2865
# rotations
seed_everything()
ml_pred = ml_test
ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=True)
ys, y_hats = predict_epochs(net, ml_pred, epochs=5)
plot_predictions(ys, y_hats)
plt.savefig(fig_folder + data_name + "_cartesian_test.pdf")

## Spherical

In [ ]:
sigma = 0.1
L = 12.8
N = 32
epochs = 50
device = get_default_device()


seed_everything()
train_df, test_df = train_test_split(df, test_size=0.1)
print(train_df.shape, test_df.shape)

seed_everything()

net = VoxelNet(nchannel)
net = net.to(device)

ml_train = MolLoader(train_df, sigma=sigma, elements=elements,
               L=L, N=N, batch_size=64, nchannel=nchannel,
               shuffle=True, rotate_randomly=True, device=device, reduce_data=True, mode='spherical')

ml_test = MolLoader(test_df, sigma=sigma, elements=elements,
               L=L, N=N, batch_size=128, nchannel=nchannel,
               shuffle=False, rotate_randomly=False, device=device, reduce_data=True, mode='spherical')

fit(epochs, net, ml_train, ml_test, torch.optim.Adam, lr=0.001, weight_decay=0.00)

### Train Plots

In [ ]:
# 0.0723
ml_pred = ml_train
ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=False)
ys, y_hats = predict(net, ml_pred)
plot_predictions(ys, y_hats)
plt.show()

In [ ]:
# 0.0669
# rotations
seed_everything()
ml_pred = ml_train
ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=True)
ys, y_hats = predict_epochs(net, ml_pred, epochs=5)
plot_predictions(ys, y_hats, alpha=0.01)
plt.show()

In [ ]:
seed_everything()
ml_wrong_mode_pred = MolLoader(train_df, sigma=sigma, elements=elements,
               L=L, N=N, batch_size=128, nchannel=nchannel,
               shuffle=False, rotate_randomly=True, device=device, reduce_data=True, mode='cartesian')
ys, y_hats = predict_epochs(net, ml_wrong_mode_pred, epochs=5)
plot_predictions(ys, y_hats, alpha=0.01)
plt.show()

### Test Plots

In [ ]:
# 0.2716
ml_pred = ml_test
ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=False)
ys, y_hats = predict(net, ml_pred)
plot_predictions(ys, y_hats)
plt.show()

In [ ]:
# 0.2629
# rotations
seed_everything()
ml_pred = ml_test
ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=True)
ys, y_hats = predict_epochs(net, ml_pred, epochs=5)
plot_predictions(ys, y_hats)
plt.savefig(fig_folder + data_name + "_spherical_test.pdf")

## Cross Validation

### Cartesian

In [ ]:
sigma = 0.1
L = 12.8
N = 32
epochs = 5
device = get_default_device()

seed_everything()
all_ys = []
all_y_hats = []

all_train_ys = []
all_train_y_hats = []

for train_indices, test_indices in KFold(n_splits=5, shuffle=True).split(df):
    
    train_df = df.iloc[train_indices]
    test_df = df.iloc[test_indices]

    net = VoxelNet(nchannel)
    net = net.to(device)
    
    ml_train = MolLoader(train_df, sigma=sigma, elements=elements,
               L=L, N=N, batch_size=64, nchannel=nchannel,
               shuffle=True, rotate_randomly=True, device=device, reduce_data=True, mode='cartesian')

    ml_test = MolLoader(test_df, sigma=sigma, elements=elements,
               L=L, N=N, batch_size=128, nchannel=nchannel,
               shuffle=False, rotate_randomly=False, device=device, reduce_data=True, mode='cartesian')
    
    fit(epochs, net, ml_train, ml_test, torch.optim.Adam, lr=0.001, weight_decay=0)
    
    ml_pred = ml_train
    ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=True)
    ys, y_hats = predict_epochs(net, ml_pred, epochs=5)
    plot_predictions(ys, y_hats)
    plt.show()
    all_train_ys.append(ys)
    all_train_y_hats.append(y_hats)
    
    ml_pred = ml_test
    ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=True)
    ys, y_hats = predict_epochs(net, ml_pred, epochs=5)
    plot_predictions(ys, y_hats)
    plt.show()
    all_ys.append(ys)
    all_y_hats.append(y_hats)

In [ ]:
plot_predictions(np.hstack(all_train_ys), np.hstack(all_train_y_hats), alpha=0.01)
plt.show()

In [ ]:
# 0.2881
plot_predictions(np.hstack(all_ys), np.hstack(all_y_hats), alpha=0.01)
plt.savefig(fig_folder + data_name + "_cartesian_cv.pdf")

### Spherical

In [ ]:
sigma = 0.1
L = 12.8
N = 32
epochs = 5
device = get_default_device()

seed_everything()
all_ys = []
all_y_hats = []

all_train_ys = []
all_train_y_hats = []

for train_indices, test_indices in KFold(n_splits=5, shuffle=True).split(df):
    
    train_df = df.iloc[train_indices]
    test_df = df.iloc[test_indices]

    net = VoxelNet(nchannel)
    net = net.to(device)
    
    ml_train = MolLoader(train_df, sigma=sigma, elements=elements,
               L=L, N=N, batch_size=64, nchannel=nchannel,
               shuffle=True, rotate_randomly=True, device=device, reduce_data=True, mode='spherical')

    ml_test = MolLoader(test_df, sigma=sigma, elements=elements,
               L=L, N=N, batch_size=128, nchannel=nchannel,
               shuffle=False, rotate_randomly=False, device=device, reduce_data=True, mode='spherical')
    
    fit(epochs, net, ml_train, ml_test, torch.optim.Adam, lr=0.001, weight_decay=0)
    
    ml_pred = ml_train
    ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=True)
    ys, y_hats = predict_epochs(net, ml_pred, epochs=5)
    plot_predictions(ys, y_hats)
    plt.show()
    all_train_ys.append(ys)
    all_train_y_hats.append(y_hats)
    
    ml_pred = ml_test
    ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=True)
    ys, y_hats = predict_epochs(net, ml_pred, epochs=5)
    plot_predictions(ys, y_hats)
    plt.show()
    all_ys.append(ys)
    all_y_hats.append(y_hats)

In [ ]:
# 0.2658
plot_predictions(np.hstack(all_ys), np.hstack(all_y_hats), alpha=0.01)
plt.savefig(fig_folder + data_name + "_spherical_cv.pdf")

## Parameter Grid Search

In [ ]:
seed_everything()
train_df, test_df = train_test_split(df, test_size=0.1)
print(train_df.shape, test_df.shape)

In [ ]:
param_grid = ParameterGrid({
    "sigma": [0.01, 0.1, 0.5, 1.],
    "L": [10, 12.8, 15],
    "weight_decay": [0., 0.001, 0.01]
})

In [ ]:
len(list(param_grid))

In [ ]:
epochs = 50
N = 32

device = get_default_device()

results = []

for i, params in enumerate(param_grid):
    L = params["L"]
    sigma = params["sigma"]
    weight_decay = params["weight_decay"]
    print(f"{i}. L = {L}, sigma = {sigma}, weight_decay = {weight_decay}")

    seed_everything()
    
    net = VoxelNet(nchannel)
    net = net.to(device)

    ml_train = MolLoader(train_df, sigma=sigma, elements=elements,
                   L=L, N=N, batch_size=64, nchannel=nchannel,
                   shuffle=True, rotate_randomly=True, device=device, reduce_data=True, mode='cartesian')

    ml_test = MolLoader(test_df, sigma=sigma, elements=elements,
                       L=L, N=N, batch_size=128, nchannel=nchannel,
                       shuffle=False, rotate_randomly=False, device=device, reduce_data=True, mode='cartesian')

    fit(epochs, net, ml_train, ml_test, torch.optim.Adam, lr=0.001, weight_decay=weight_decay, verbose=False)
    
    ml_pred = ml_test
    ml_pred.reset(batch_size=128, shuffle=False, rotate_randomly=True)
    ys, y_hats = predict_epochs(net, ml_pred, epochs=5)
    plot_predictions(ys, y_hats)
    
    results.append(np.mean((ys - y_hats)**2))

In [ ]:
for param, mse in zip(param_grid, results):
    print(param, mse)

In [ ]:
amin = np.argmin(results)
param_grid[amin], results[amin]